In [45]:
# Imports

from math import *
import pandas as pd
from scipy.interpolate import RegularGridInterpolator as RGI
import numpy as np
import matplotlib.pyplot as plt
import os

print("Esperamos trabalhar no diretório: "+str(os.getcwd()))

Esperamos trabalhar no diretório: C:\Users\henriquegos\Desktop\Python


In [46]:
# Data

data = pd.read_excel("C:/Users/henriquegos/Desktop/VPDTABLEPASTA/VPDTABLE.xlsx")

# Limpeza da tabela

data = data.iloc[3:24,0:14]
data.columns = ["Temperatura\Humidade","35%","40%","45%","50%","55%","60%","65%","70%","75%","80%","85%","90%","95%"]

#Tranformando os valores de VPD em vetores
VPD = data.iloc[1:21,1:14].to_numpy()

#Eixo Y e EixoVPD
TemperaturaDados = np.arange(15,36,1)

diffs = 21
i_temp = 0

Humidade, Temperatura = float(input("Qual a humidade do seu grow?")), float(input("Qual a temperatura do seu grow?"))

for i, temp in enumerate(TemperaturaDados):
                         
    
    diffscalc = sqrt((Temperatura-temp)**2)                  
    if diffscalc < diffs:
        diffs = diffscalc
        i_temp = i
         

if i_temp-6<0:
    
    TemperaturaDados = TemperaturaDados[0:13]
    VPD = VPD[0:13]
    
elif i_temp+6>19:
    
    TemperaturaDados = TemperaturaDados[8:21]
    VPD = VPD[7:20]
    
else:
    
    TemperaturaDados = TemperaturaDados[i_temp-6:i_temp+7]
    VPD = VPD[i_temp-6:i_temp+7]

#Eixo X
HumidadeDados = np.arange(35,96,5)


Qual a humidade do seu grow?50.7
Qual a temperatura do seu grow?25.9


In [47]:
#Realizando a interpolação
f = RGI((HumidadeDados, TemperaturaDados), VPD, method = "linear", fill_value = -1)

print("Seu déficit de pressão de vapor é: {:.3}".format(f((Humidade,Temperatura))))

Seu déficit de pressão de vapor é: 1.07
